In [1]:
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import glob
import os

# Parse NYT pages

In [2]:
# waybackpack https://www.nytimes.com/ -d test-wp-wayback --from-date 2022100101 --to-date 2022100112

In [3]:
nyt_pages = glob.glob('../test-scraping-methods/test-nytimes-waybackpack/*/*/*')

In [4]:
fname = nyt_pages[0]
with open(fname) as f:
    html = f.read()

In [5]:
soup = BeautifulSoup(html)

head = soup.head
head.append(soup.new_tag('style', type='text/css'))
head.style.append('.story-wrapper { border: 4px dotted blue; }')

In [6]:
display(HTML(str(soup)))

# Parse Washington Post pages

In [7]:
# waybackpack https://www.washingtonpost.com/ -d test-wp-wayback --from-date 2022100101 --to-date 2022100112

In [25]:
wp_pages = glob.glob('../test-scraping-methods/test-wp-waybackpack/*/*/*')
file_sizes = list(map(os.path.getsize, wp_pages))
wp_pages = list(filter(lambda x: x[1] > 0, zip(wp_pages, file_sizes)))
wp_pages = list(map(lambda x: x[0], wp_pages))

In [30]:
wp_pages

['../test-scraping-methods/test-wp-waybackpack/20221001083754/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001033238/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001043334/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001041031/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001074544/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001021634/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001043007/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001121530/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001104316/www.washingtonpost.com/index.html',
 '../test-scraping-methods/test-wp-waybackpack/20221001110141/www.washingtonpost.com/index.html',
 '../test-scraping-m

In [26]:
fname = wp_pages[0]
with open(fname) as f:
    html = f.read()

In [28]:
soup = BeautifulSoup(html)

In [32]:
! curl 'http://web.archive.org/cdx/search/cdx?url=https://www.washingtonpost.com&fl=urlkey,timestamp' > urls.txt ;

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.9M    0 25.9M    0     0  2159k      0 --:--:--  0:00:12 --:--:-- 2778k


# Look at all pages 

In [41]:
import pandas as pd 
sites_df = pd.read_csv('../bin/news-homepages/newshomepages/sources/sites.csv')

In [45]:
t = sites_df['url'].iloc[-1]

In [101]:
sites_df['url'].iloc[0]

'https://www.wbtv.com/'

In [51]:
t

'https://www.charlotteobserver.com/'

In [53]:
import subprocess

In [82]:
from subprocess import Popen, PIPE

p = Popen(['ls', '-lh'], stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = p.communicate()
rc = p.returncode

In [102]:
t3 = ["waybackpack", t,
    "--from-date",
    "2023010100",
    "--to-date",
    "2023010200",
    "--list"]

In [104]:
' '.join(t3)

'waybackpack https://www.charlotteobserver.com/ --from-date 2023010100 --to-date 2023010200 --list'

In [94]:
output, err = Popen([
    "waybackpack", t,
    "--from-date",
    "2023010100",
    "--to-date",
    "2023010200",
    "--list"
], stdin=PIPE, stdout=PIPE, stderr=PIPE).communicate()

output.decode().split()

In [ ]:
subprocess.call()

In [81]:
output

b'\n'

In [48]:
urls = ! waybackpack $t --from-date 2023010100 --to-date 2023010200 --list

In [52]:
urls[0]

'https://web.archive.org/web/20230101002101/https://www.charlotteobserver.com/'

In [ ]:
%%bash 
wget \
    --no-clobber \
    --page-requisites \
    --convert-links \
    --domains web.archive.org \
    --no-parent https://web.archive.org/web/20230101002101/https://www.charlotteobserver.com/

## Download as Single-File

### try with selenium

In [245]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.support.ui import WebDriverWait
import pyautogui

URL = 'https://en.wikipedia.org/wiki/Python_(programming_language)'
URL = "https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/"
FILE_NAME = 'test-wp-output.mhtml'

# open page with selenium
# (first need to download Chrome webdriver, or a firefox webdriver, etc)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path='/Users/alex/chromedriver', options=options)
driver.get(URL)

if False:
    # wait until body is loaded
    WebDriverWait(driver, 60).until(visibility_of_element_located((By.TAG_NAME, 'body')))
    time.sleep(1)
    # open 'Save as...' to save html and assets
    pyautogui.hotkey('ctrl', 's')
    time.sleep(1)
    if FILE_NAME != '':
        pyautogui.typewrite(FILE_NAME)
    pyautogui.hotkey('enter')

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  from ipykernel import kernelapp as app


### try with playwright

In [17]:
import playwright
from path import Path
from playwright.sync_api import sync_playwright
import tempfile

width = 1300
height = 1600
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"    
    

THIS_DIR = Path('/Users/alex/Projects/usc-research/newsworthiness/bin/news-homepages/newshomepages/').parent#.absolute()
EXTENSIONS_PATH = THIS_DIR / "extensions"

extensions_list = [
    EXTENSIONS_PATH / "adguard",
    EXTENSIONS_PATH / "adguardextra",
]
f1 = 'single-file-cli/lib/single-file-bootstrap.js'
f2 = 'single-file-cli/lib/single-file-hooks-frames.js'
f3 = 'single-file-cli/lib/single-file-frames.js'
f4 = 'single-file-cli/lib/single-file.js'

extensions_str = ",".join(map(str, extensions_list))

def read_script_from_file(filename, page_obj):
    with open(filename) as f:
        script = f.read()
        page_obj.evaluate(script)

with sync_playwright() as p:
    data_dir = tempfile.mkdtemp()
    context = p.chromium.launch_persistent_context(
        data_dir,
        channel="chrome",
        headless=False,
        args=[
            f"--disable-extensions-except={extensions_str}",
            f"--load-extension={extensions_str}",
            "--disable-gpu",
            "--disable-notifications",
            "--disable-search-geolocation-disclosure",
            "--enable-logging=stderr",
            "--log-level=0",
            "--v=1",
        ],
        user_agent=user_agent,
        viewport={
            "width": width,
            "height": height,
        },
    )
    
    page = context.new_page()
    for fname in [f1, f2, f3]:
        read_script_from_file(fname, page)
    
    page.goto("http://playwright.dev")
    read_script_from_file(f4, page)
    
    runner_script = """
    () => singlefile.getPageData({
          removeHiddenElements: true,
          removeUnusedStyles: true,
          removeUnusedFonts: true,
          removeImports: true,
          blockScripts: true,
          blockAudios: true,
          blockVideos: true,
          compressHTML: true,
          removeAlternativeFonts: true,
          removeAlternativeMedias: true,
          removeAlternativeImages: true,
          groupDuplicateImages: true
        });
    """
    output = page.evaluate(runner_script)
    html_content = output['content']
    print(html_content[:500])
    context.close()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

# Run Playwright manually

In [ ]:
def _get_common_blocking_javascript() -> str:
    """Compiles a set of commands into an executable javascript script to block common pop-ups, banners, etc."""
    # Run common JavaScript for all sites
    target_list = [
        ".tp-modal",  # Common popover ad
        ".tp-backdrop",
        ".onesignal-slidedown-container",  # Common slidedown ad
        "#regiwall-overlay",
        ".regiwall",
        ".bxc",  # Common takeover ad
        ".met-footer-toast",  # Common toaster at the bottom
        ".grecaptcha-badge",
        ".fEy1Z2XT",  # Common ad blocker popup
        ".dgEhJe6g",
        ".notification-soft-optin",  # Common notification popup
        ".popup_background",
        ".popup_wrapper",
        ".pw-subscribe-popup",
        ".ThirdPartySlot-container",
        ".newspack-lightbox",  # Common takover ad
        ".pbs_loc",  # PBS site location picker
        "#onetrust-consent-sdk",
        ".gdpr-huffpost-cookiewall",
        "#header-cts",
        ".pum-overlay",  # Common takeover ad
        "#pico_prompt",
        "#pico_header",
        ".mol-ads-cmp",  # Footer toaster thing on some sites
        ".gdpr-glm-standard",  # GDPR blockers on some sites
        "#didomi-host",  # Common cookies popup
        ".fc-ab-root",  # Ad blocker popup
        ".fancybox-overlay",  # Popup overlay
        ".fancybox-overlay-fixed",
        ".ab_widget_container_popin-image",  # Popover
        ".ab_widget_container_popin-image_content",
        ".ab_widget_container_popin-image_overlay",
        ".fc-ab-root",  # Popover
        ".modal-backdrop",  # Modal popup
        "#dth-borderless-modal-alt",
        "#floatingBlockerGuide",  # Popup overlay
        ".ctct-popup-overlay",  # Popup overlay
        ".ctct-popup-wrapper",
        ".message-container",  # Popup
        ".teg-container",
        'div[id^="sp_message_container"]',  # Popover
        ".wisepops-root",  # Popover
        "#wisepops-root",
        ".widget_eu_cookie_law_widget",  # GDPR poup
        ".fancybox-overlay",  # Ad overlay
        ".signup-box",  # Popup ad
        ".newspack-lightbox",  # Popup
        ".adunit-googleadmanager",  # Google ad
        ".mc-modal",  # Newsletter popup
        ".mc-modal-bg",
        ".blockNavigation",
        ".ab-iam-root",
        ".omaha-background",  # Takeover ad
        ".sqs-popup-overlay",  # Overlay popup
        "#gcomPromo",  # Overlay found on Brazilian sites
        ".Campaign",
    ]
    target_str = ",".join(target_list)
    return f"""
        document.querySelectorAll('{target_str}').forEach(el => el.remove());
        var styleSheet = document.createElement('style');
        styleSheet.innerText = '{target_str} {{ display: none !important; }}';
        document.head.appendChild(styleSheet);
     """

import playwright
from path import Path
from playwright.sync_api import sync_playwright
import tempfile
import time 

width = 1300
height = 1600
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"    

THIS_DIR = Path('/Users/alex/Projects/usc-research/newsworthiness/bin/news-homepages/newshomepages/').parent#.absolute()
EXTENSIONS_PATH = THIS_DIR / "extensions"

extensions_list = [
    EXTENSIONS_PATH / "adguard",
    EXTENSIONS_PATH / "adguardextra",
]
extensions_str = ",".join(map(str, extensions_list))

with sync_playwright() as p:
    data_dir = tempfile.mkdtemp()
    # context = sync_playwright().start().chromium.launch_persistent_context(
    context = p.chromium.launch_persistent_context(
        data_dir,
        channel="chrome",
        headless=False,
        args=[
            f"--disable-extensions-except={extensions_str}",
            f"--load-extension={extensions_str}",
            "--disable-gpu",
            "--disable-notifications",
            "--disable-search-geolocation-disclosure",
            "--enable-logging=stderr",
            "--log-level=0",
            "--v=1",
        ],
        user_agent=user_agent,
        viewport={
            "width": width,
            "height": height,
        },
    )
    print("Waiting 15 seconds for AdGuard filters to load")
    time.sleep(10)
    wait_seconds = 5
    page = context.new_page()    
    page.goto("https://www.chicagotribune.com/")
    print(f"Waiting {wait_seconds} seconds")
    time.sleep(wait_seconds)
    common_javascript = _get_common_blocking_javascript()
    print("Executing common JavaScript")
    page.evaluate(common_javascript)
    page.evaluate('location.reload()')
    
    def _execute_on_ready(js_code_to_execute, page, add_return=False):
        if add_return:
            js_code_to_execute = 'return ' + js_code_to_execute
        js = """(function(){
                    if(document.readyState === 'ready' || document.readyState === 'complete') {
                        %(js_code)s
                    } else {
                        return 'not ready'
                        }
            })()
        """ % {'js_code': js_code_to_execute}
        print(js)
        res = page.evaluate(js)
        while res == 'not ready':
            print('not ready....')
            time.sleep(1)
            res = page.evaluate(js)
        return res
    
    res = _execute_on_ready('''document.body.style.overflow = 'hidden';''', page)
    print(res)

# Add test sites for full-page screenshot

### Download as SingleFile

In [252]:
f1 = 'lib/single-file-bootstrap.js'
f2 = 'lib/single-file-hooks-frames.js'
f3 = 'lib/single-file-frames.js'

f4 = 'lib/single-file.js'

In [253]:
'''const { content, title, filename } = await singlefile.getPageData({
  removeHiddenElements: true,
  removeUnusedStyles: true,
  removeUnusedFonts: true,
  removeImports: true,
  blockScripts: true,
  blockAudios: true,
  blockVideos: true,
  compressHTML: true,
  removeAlternativeFonts: true,
  removeAlternativeMedias: true,
  removeAlternativeImages: true,
  groupDuplicateImages: true
});'''

'const { content, title, filename } = await singlefile.getPageData({\n  removeHiddenElements: true,\n  removeUnusedStyles: true,\n  removeUnusedFonts: true,\n  removeImports: true,\n  blockScripts: true,\n  blockAudios: true,\n  blockVideos: true,\n  compressHTML: true,\n  removeAlternativeFonts: true,\n  removeAlternativeMedias: true,\n  removeAlternativeImages: true,\n  groupDuplicateImages: true\n});'

In [ ]:
from selenium import webdriver
#set chromodriver.exe path
driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
driver.maximize_window()
#launch URL
driver.get("https://www.tutorialspoint.com/index.htm")
#get window size
s = driver.get_window_size()
#obtain browser height and width
w = driver.execute_script('return document.body.parentNode.scrollWidth')
h = driver.execute_script('return document.body.parentNode.scrollHeight')
#set to new window size
driver.set_window_size(w, h)
#obtain screenshot of page within body tag
driver.find_element_by_tag_name('body').screenshot("tutorialspoint.png")
driver.set_window_size(s['width'], s['height'])
driver.quit()

In [200]:
! ../single-file-cli/single-file \
    --back-end webdriver-chromium \
        https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/ test.html \
    --browser-load-max-time 180000

In [11]:
with open('test.html') as f:
    html = f.read()

In [12]:
soup = BeautifulSoup(html)
head = soup.head
head.append(soup.new_tag('style', type='text/css'))
head.style.append('.card { border: 4px dotted blue; }')

In [28]:
# works !! 
# display(HTML(str(soup)))

In [251]:
# driver('file://Users/alex/Projects/usc-research/newsworthiness/notebooks/test.html')

In [25]:
with open('../bin/news-homepages/washingtonpost.html') as f :
    nytimes_fullpage_html = f.read()

In [34]:
# display(HTML(nytimes_fullpage_html))

In [29]:
import imgkit

In [31]:
# imgkit.from_file('../bin/news-homepages/washingtonpost.html', 'out.jpg')
from html2image import Html2Image
hti = Html2Image()

In [32]:
hti.screenshot(html_str=nytimes_fullpage_html, save_as='page.png', )

['/Users/alex/Projects/usc-research/newsworthiness/notebooks/page.png']

In [33]:
! open page.png

### Find Bounds of DOM Objects Through Selenium

In [211]:
from selenium.webdriver.common.by import By

In [237]:
es = driver.find_elements(by=By.CLASS_NAME, value='card')

In [238]:
e = es[0]

In [239]:
location = e.location
size = e.size
w, h = size['width'], size['height']

In [240]:
size

{'height': 625, 'width': 1024}

In [241]:
w

1024

In [242]:
h

625

In [219]:
driver.save_screenshot('test.png')

True

In [222]:
! open 'test.png'

### Parse as MHTML

In [23]:
res = driver.execute_cdp_cmd('Page.captureSnapshot', {})

# 2. write file locally
with open('chrome-wp-test.mhtml', 'w', newline='') as f:
    f.write(res['data'])

In [3]:
import smhtml
smhtml.extract('chrome-wp-test.mhtml', 'test', usebasename=False, outputfilenamer=None)

In [4]:
ls

2022-10-12__analyze-wayback-machine.ipynb
chrome-wp-test.mhtml
index.html
test/


In [28]:
html = driver.page_source

In [37]:
import mhtml, mhtml_scripts

In [42]:
from mhtml_scripts import extract_main

In [46]:
t = res['data'].encode()

In [47]:
mhtml.parse_mhtml(t)

(ResourceHeader: [('From', '<Saved by Blink>\r'), ('napshot-Content-Location', 'https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/\r'), ('ubject', 'The Washington Post - Breaking news and latest headlines, U.S. news, world news, and video\r'), ('ate', 'Thu, 13 Oct 2022 03:34:58 -0000\r'), ('IME-Version', '1.0\r'), ('ontent-Type', 'multipart/related;\r')],
 None)

In [48]:
mhtml.parse_header(t, 0 )

(ResourceHeader: [('From', '<Saved by Blink>\r'), ('napshot-Content-Location', 'https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/\r'), ('ubject', 'The Washington Post - Breaking news and latest headlines, U.S. news, world news, and video\r'), ('ate', 'Thu, 13 Oct 2022 03:34:58 -0000\r'), ('IME-Version', '1.0\r'), ('ontent-Type', 'multipart/related;\r')],
 429)

In [ ]:
header = ResourceHeader()
next_pos = from_pos

num_empty_lines = 0
while next_pos != -1:
    line, next_pos = next_line(content, next_pos)

    if len(line) <= 2:
        num_empty_lines += 1
        break

    line = line[:-2]
    line = line.decode()

    parts = line.split(': ', 1)
    if len(parts) != 2:
        logger.warning('No separator in line: %s', line)
        continue

    header[parts[0]] = parts[1]

assert num_empty_lines >= 1, 'after header at least one empty line!'

In [ ]:
pos = 0
headers, header_end_pos = mhtml.parse_header(t, 0)
line, next_pos = mhtml.next_line(t, header_end_pos)
if len(line) != 2:
    print('After main header should follow two empty lines?, %d', header_end_pos)
else:
    header_end_pos = next_pos
print('Header: %d -- %d: %s', pos, header_end_pos, headers)

boundary = mhtml.get_boundary(headers)
# if boundary is None:
#     print('Found no boundary in header!')
#     return headers, None
# logger.debug('Bounary: %s', boundary[2:])

# parse body parts ...
parts, parts_end_pos = mhtml.parse_parts(t, boundary, header_end_pos)
logger.debug('Got %d parts.', len(parts))
assert parts_end_pos == -1, 'file should be completly parsed'

In [56]:
headers

ResourceHeader: [('From', '<Saved by Blink>\r'), ('napshot-Content-Location', 'https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/\r'), ('ubject', 'The Washington Post - Breaking news and latest headlines, U.S. news, world news, and video\r'), ('ate', 'Thu, 13 Oct 2022 03:34:58 -0000\r'), ('IME-Version', '1.0\r'), ('ontent-Type', 'multipart/related;\r')]

In [96]:
t2 = t.decode('utf-8').split('----MultipartBoundary--')

In [97]:
t2[0]

'From: <Saved by Blink>\r\nSnapshot-Content-Location: https://web.archive.org/web/20221001004532id_/https://www.washingtonpost.com/\r\nSubject: The Washington Post - Breaking news and latest headlines, U.S. news, world news, and video\r\nDate: Thu, 13 Oct 2022 03:34:58 -0000\r\nMIME-Version: 1.0\r\nContent-Type: multipart/related;\r\n\ttype="text/html";\r\n\tboundary="'

In [107]:
from collections import defaultdict
content_dict = defaultdict(list)

In [150]:
html = ''.join(content_dict['text/html'])
soup = BeautifulSoup(html)

In [146]:
head = soup.head
head.append(soup.new_tag('style', type='text/css'))
head.style.append(''.join(content_dict['text/css']))

In [154]:
from pywebcopy import save_webpage

In [156]:
download_folder = '/path/to/downloads/'    
kwargs = {'bypass_robots': True, 'project_name': 'recognisable-name'}
save_webpage(URL, 'test-pywebcopy/', **kwargs)


KeyboardInterrupt



In [65]:
from_pos = 0
header = mhtml.ResourceHeader()
next_pos = from_pos

num_empty_lines = 0
while next_pos != -1:
    line, next_pos = mhtml.next_line(t, next_pos)

    if len(line) <= 2:
        num_empty_lines += 1
        break

    line = line[:-2]
    line = line.decode()

    parts = line.split(': ', 1)
    if len(parts) != 2:
        print('No separator in line: %s' % line)
        continue

    header[parts[0]] = parts[1]

assert num_empty_lines >= 1, 'after header at least one empty line!'

No separator in line: type="text/html";
No separator in line: boundary="----MultipartBoundary--lZZbHqjQD8mGaM5rSaV5CPwHrHZajUWW1Qnh2be9w2----"


In [44]:
extract_main.main('chrome-wp-test.mhtml')

TypeError: 'NoneType' object is not iterable

# Download all asset files

In [33]:
import os 

In [12]:
archived_files = ! waybackpack https://www.nytimes.com/ --list --from-date 20221001 --to-date 20221002

In [16]:
! wget --help

GNU Wget 1.21.2, a non-interactive network retriever.
Usage: wget [OPTION]... [URL]...

Mandatory arguments to long options are mandatory for short options too.

Startup:
  -V,  --version                   display the version of Wget and exit
  -h,  --help                      print this help
  -b,  --background                go to background after startup
  -e,  --execute=COMMAND           execute a `.wgetrc'-style command

Logging and input file:
  -o,  --output-file=FILE          log messages to FILE
  -a,  --append-output=FILE        append messages to FILE
  -d,  --debug                     print lots of debugging information
  -q,  --quiet                     quiet (no output)
  -v,  --verbose                   be verbose (this is the default)
  -nv, --no-verbose                turn off verboseness, without being quiet
       --report-speed=TYPE         output bandwidth as TYPE.  TYPE can be bits
  -i,  --input-file=FILE           download URLs found in local or external FILE
  

In [26]:
t = glob.glob('../test-nytimes-waybackpack/*/*/*')

In [38]:
file_sizes = list(map(os.path.getsize, t))

In [39]:
file_sizes[0]

2652486

In [42]:
from bs4 import BeautifulSoup

In [46]:
with open(t[0]) as f:
    soup = BeautifulSoup(f.read())

In [51]:
imgs = soup.find_all('img')

In [68]:
imgs[2].attrs['src']

'https://web.archive.org/web/20221001161447im_/https://static01.nyt.com/images/2022/09/30/autossell/swift_flooding-1000w-1000h/swift_flooding-1000w-1000h-square320.jpg'

In [69]:
! wget -x -P test $imgs[2].attrs['src']

/bin/sh: img: No such file or directory


In [62]:
! wget --help

GNU Wget 1.21.2, a non-interactive network retriever.
Usage: wget [OPTION]... [URL]...

Mandatory arguments to long options are mandatory for short options too.

Startup:
  -V,  --version                   display the version of Wget and exit
  -h,  --help                      print this help
  -b,  --background                go to background after startup
  -e,  --execute=COMMAND           execute a `.wgetrc'-style command

Logging and input file:
  -o,  --output-file=FILE          log messages to FILE
  -a,  --append-output=FILE        append messages to FILE
  -d,  --debug                     print lots of debugging information
  -q,  --quiet                     quiet (no output)
  -v,  --verbose                   be verbose (this is the default)
  -nv, --no-verbose                turn off verboseness, without being quiet
       --report-speed=TYPE         output bandwidth as TYPE.  TYPE can be bits
  -i,  --input-file=FILE           download URLs found in local or external FILE
  